# **Regularizaztion**
**Recap**
-






In [74]:
from tensorflow.keras.layers import Dropout, Softmax
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential, Model
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model


In [75]:
data = pd.read_csv("mnist_train.csv")
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:

X = data.iloc[:, 1:] / 255.0   # normalize
y = data.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

In [69]:
def build_model_l1(rate=0.001):
    model = Sequential()
    model.add(Dense(128, activation="relu", input_shape=(784,),
                    kernel_regularizer = regularizers.L1(rate)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [71]:
def build_model_l2(rate=0.001):
    model = Sequential()
    model.add(Dense(128, activation="relu", input_shape=(784,),
                    kernel_regularizer=regularizers.L2(rate)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [72]:
def build_model_dropout(rate=0.2):
    model = Sequential()
    model.add(Dense(128, activation="relu", input_shape=(784,)))
    model.add(Dropout(rate))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [78]:

model_l1 = build_model_l1()
model_l1.fit(X_train, y_train_cat, epochs=5, batch_size=128, verbose=0)
l1_acc = model_l1.evaluate(X_test, y_test_cat, verbose=0)[1]
l1_error = 1 - l1_acc

model_l2 = build_model_l2()
model_l2.fit(X_train, y_train_cat, epochs=5, batch_size=128, verbose=0)
l2_acc = model_l2.evaluate(X_test, y_test_cat, verbose=0)[1]
l2_error = 1 - l2_acc

model_dropout = build_model_dropout(0.2)
model_dropout.fit(X_train, y_train_cat, epochs=5, batch_size=128, verbose=0)
dropout_acc = model_dropout.evaluate(X_test, y_test_cat, verbose=0)[1]
dropout_error = 1 - dropout_acc

results = pd.DataFrame({
    "Model": ["DNN + L1", "DNN + L2", "DNN + Dropout(0.2)"],
    "Error Rate": [l1_error, l2_error, dropout_error]
})
results


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,Model,Error Rate
0,DNN + L1,0.067000
1,DNN + L2,0.034083
2,DNN + Dropout(0.2),0.030167


## Callback
- Early Stopping
- Model Checkppoint
- Learning Rate Scheduler
- TensorBoard
- Custom History


In [119]:

from tensorflow.keras.callbacks import Callback
class EarlyStopper(Callback):
  def __init__(self, target_acc=None, target_loss=None):
    super(EarlyStopper, self).__init__()
    self.target_acc = target_acc
    self.target_loss = target_loss
  def on_epoch_end(self, epoch, logs={}):
    acc = logs['val_accuracy']
    loss = logs['val_loss']
    if acc>= self.target_acc:
      self.model.stop_training = True
    if loss<=self.target_loss:
      self.model.stop_training = True


In [120]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [121]:
early_stopper = EarlyStopper(target_acc=0.95, target_loss=0.1)

In [122]:
history = model.fit(X_train, y_train_cat, epochs=10, batch_size=32, validation_data=(X_test, y_test_cat), callbacks=[early_stopper])

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9748 - loss: 0.0813 - val_accuracy: 0.9742 - val_loss: 0.0882


In [123]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = "Dnn_model_val_acc_{val_accuracy:.0f}.keras"

check_point = ModelCheckpoint(
    filepath=filepath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)